## Transportation Forecasting
For TRBAM 2019 TRANSFOR19 Forecasting competition

notes:

- total data size = `(17856 rows × 1024 columns)`

- available date range = `2016-10-01 to 2016-12-01`

- speed forecasting range = `2016-12-01 00:00:00+08:00 to 2016-12-01 23:55:00+08:00` `6am - 10:55am, 4pm to 8:55pm`

- prediction boundaries: 
  * `108.94615156073496 < longitude < 108.94765628015638`
  * `34.2324012260476 < Latitude < 34.23940650580562`

- Dec 01, 2016 is a Thursday

### This is Part 1 of 3 part series:

Use the following to navigate

- [Part 1: data processing](01_processing.ipynb)
- [Part 2: data preparation](02_preparation.ipynb)
- [Part 3: model training](03_training.ipynb)

-----

## 1. Setup and library installation

Requirements: 
- Python >= 3.5 (3.7 recommended)
- BLAS libraries (OpenBLAS, ATLAS, Intel MKL etc.) 
- gcc >= 4.9
- Git
- Python pip

example install on Debian Linux:

`apt install python3.7 python3.7-dev python3-pip g++ libblas-dev git`

to install basic required Python libraries run:

`pip3 install --user -r requirements.txt`

## 2. Data Analysis

Python script

In [1]:
# import Python libraries

import sys
import os
import io
import tarfile 
import time
import pytz
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
# setup the default parameters

lon_min = 108.911189
lon_max = 108.99861
lon_range = lon_max - lon_min

lat_min = 34.2053
lat_max = 34.280221
lat_range = lat_max - lat_min

n_segments = 32

print(
    'Longitude range: {}\n'
    'Latitude range: {}\n'.format(lon_range, lat_range)
)

Longitude range: 0.08742100000000619
Latitude range: 0.07492099999999624



In [3]:
# create the empty base table that we are inputing values from the datasets
time_range = np.arange('2016-10-01 00:00:00', '2016-12-02 00:00:00', np.timedelta64(5, 'm'), dtype='datetime64')
col_range = np.arange(0, n_segments**2)

# construct the pandas DataFrame object
datatable = pd.DataFrame(index=time_range, columns=col_range).fillna(0.)
datatable.index.name = 'Timestamp'
datatable.index = datatable.index.tz_localize(pytz.timezone('Asia/Shanghai'))

speedtable = pd.DataFrame(index=time_range, columns=['speed_north', 'speed_south'])
speedtable.index.name = 'Timestamp'
speedtable.index = speedtable.index.tz_localize(pytz.timezone('Asia/Shanghai'))

# check and display
assert datatable.index.dtype == 'datetime64[ns, Asia/Shanghai]'
assert speedtable.index.dtype == 'datetime64[ns, Asia/Shanghai]'

display(speedtable.head())
display(datatable.head())

,speed_north,speed_south
Timestamp,,
2016-10-01 00:00:00+08:00,NaN,NaN
2016-10-01 00:05:00+08:00,NaN,NaN
2016-10-01 00:10:00+08:00,NaN,NaN
2016-10-01 00:15:00+08:00,NaN,NaN
2016-10-01 00:20:00+08:00,NaN,NaN


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
Timestamp,,,,,,,,,,,,,,,,,,,,,
2016-10-01 00:00:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:05:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:10:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:15:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:20:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# save the empty table as a template
datatable.to_csv('datatable_empty.csv')
speedtable.to_csv('speedtable_empty.csv')

-----

## 3. Import and extract data from the GAIA dataset

- enumerate over all the \*.tar.gz files 
- process using io.BytesIO() (reads directly from *.tar.gz)
- create a list of files `tarfile_ls[]`

In [14]:
# run through the directories and generate a list of filenames
month_folder = ['october', 'november', 'december']
tarfile_ls = []

# iterate through each folder and generate a list of files
for month_name in month_folder:
    for root, dirs, files in os.walk(os.path.join(month_name)):
        for n, f in enumerate(files, 1):
            tf = tarfile.open(os.path.join(month_name, f))
            assert len(tf.members) == 1  # check if there is only one file
            
            # add file member to list
            tarfile_ls.append(tf)
            
            print(
                '{0:d}: {1} {2}\n'
                '    in {3}\n'.format(n, os.path.join(month_name, f), tf.members[0], tf))


1: october/gps_20161026.tar.gz <TarInfo 'xian/gps_20161026' at 0x7f02155b89a8>
    in <tarfile.TarFile object at 0x7f024752f710>

2: october/gps_20161027.tar.gz <TarInfo 'xian/gps_20161027' at 0x7f02155b85c0>
    in <tarfile.TarFile object at 0x7f024752f6a0>

3: october/gps_20161028.tar.gz <TarInfo 'xian/gps_20161028' at 0x7f02155b8b38>
    in <tarfile.TarFile object at 0x7f024752fa58>

4: october/gps_20161029.tar.gz <TarInfo 'xian/gps_20161029' at 0x7f02155b8c00>
    in <tarfile.TarFile object at 0x7f024752f860>

5: october/gps_20161001.tar.gz <TarInfo 'xian/gps_20161001' at 0x7f02155b8cc8>
    in <tarfile.TarFile object at 0x7f024752f828>

6: october/gps_20161002.tar.gz <TarInfo 'xian/gps_20161002' at 0x7f02155b8d90>
    in <tarfile.TarFile object at 0x7f024752fef0>

7: october/gps_20161003.tar.gz <TarInfo 'xian/gps_20161003' at 0x7f02155b8e58>
    in <tarfile.TarFile object at 0x7f024752fba8>

8: october/gps_20161004.tar.gz <TarInfo 'xian/gps_20161004' at 0x7f02155b8f20>
    in <tar

- For each \*.tar.gz data file, compute the number of users (drivers) on the road 
- Input the number of observations e.g. `groupby` into a 32 x 32 cell matrix.
- Each cell is calculated over every 5 minute interval
- Cell matrix is transformed into 1024 columns
- Haversine distance formula:

\begin{equation}
2r\arcsin\Big(\sqrt{\sin^2(\frac{\phi_2-\phi_1}{2})+\cos(\phi_1)\cos(\phi_2)\sin^2(\frac{\theta_2-\theta_1}{2})}\Big)
\end{equation}

In [16]:
# functions for speed and distance calculations

def calculate_distance(lat1, lon1, lat2, lon2):
    # haversine distance
    earth_radius = 6371  # km
    dlat = np.radians(lat2-lat1)
    dlon = np.radians(lon2-lon1)
    a = np.sin(dlat/2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) ** 2
    distance = 2 * earth_radius * np.arcsin(np.sqrt(a))
    return distance * 1000  # meters

def calculate_speed(x):
    d = {}
    d['Timestamp_begin'] = pd.to_datetime(x['Timestamp'].astype(int).min())
    d['Speed_mean'] = x['Speed'].mean()
    return pd.Series(d, index=['Timestamp_begin', 'Speed_mean'])

- Algorithm to process each datefile in the GAIA dataset from October 1st to December 1st
- Speed data is calculated using the haversine distance and interval between each observation

In [18]:
assert datatable.sum().sum() == 0  # check is datatable is empty

start_time = time.time()
for n, tf in enumerate(tarfile_ls, 1):

    # read data directly from inside the tar file
    with io.BytesIO(tf.extractfile(tf.members[0]).read()) as xian_gps_info:
        df = pd.read_csv(
            filepath_or_buffer=xian_gps_info, 
            sep=',', 
            header=None, 
            names=['Driver_ID', 'Order_ID', 'Timestamp', 'Longitude', 'Latitude']
        )
        
        # convert timestamp to datetime dtype
        df['Timestamp'] = pd.to_datetime(df['Timestamp'].values, unit='s')
        
        # assign segment numbers to each observation
        df['Longitude_seg'] = np.floor((df['Longitude'].values-lon_min)/lon_range*n_segments).astype('int')
        df['Latitude_seg'] = np.floor((df['Latitude'].values-lat_min)/lat_range*n_segments).astype('int')
        df['segment_num'] = df['Longitude_seg'].values*n_segments + df['Latitude_seg'].values
        df.loc[
            (df['Longitude']>108.94615156073496) & (df['Longitude']<108.94765628015638) & 
            (df['Latitude']>34.2324012260476) & (df['Latitude']<34.23940650580562), 'zone_interest'
        ] = 1.
        df['zone_interest'] = df['zone_interest'].fillna(0.)
        
        #################
        # Process speed #
        #################
        
        # compute speed per observation in road segment
        df_roadsection = df.loc[df['zone_interest']==1.]
        df_roadsection = df_roadsection.sort_values(['Driver_ID', 'Order_ID', 'Timestamp'])
        df_roadsection[['Time_delta', 'Longitude_delta', 'Latitude_delta']] = df_roadsection.sort_values(
            ['Driver_ID', 'Order_ID', 'Timestamp']).loc[:, ['Timestamp', 'Longitude', 'Latitude']].diff()
        
        # locate and ignore points where [Driver_ID, Order_ID] changes or between trajectories
        df_roadsection.loc[
            (
                (df_roadsection.loc[:, ['Time_delta']] > pd.Timedelta('9 sec')) | 
                (df_roadsection.loc[:, ['Time_delta']] < pd.Timedelta('1 sec'))  | 
                (pd.isnull(df_roadsection.loc[:, ['Time_delta']]))
            ).values.flatten(), ['Time_delta']] = np.nan
        
        df_roadsection['Time_delta'] = df_roadsection['Time_delta'].dt.total_seconds()
        df_roadsection['Speed'] = calculate_distance(
            0, 0, 
            df_roadsection['Latitude_delta'], df_roadsection['Longitude_delta']
        ) / df_roadsection['Time_delta'] * 3.6

        # infer direction from Latitude change: -ve == traveling south, +ve == traveling north
        conditions = [
            ((df_roadsection['Latitude_delta'] == 0.) & 
             (df_roadsection['Speed'] == 0.) & 
             (df_roadsection['Time_delta'] > 0.)), 
            ((df_roadsection['Latitude_delta'] < 0.) & 
             (df_roadsection['Time_delta'] > 0.)), 
            ((df_roadsection['Latitude_delta'] > 0.) & 
             (df_roadsection['Time_delta'] > 0.))
        ]
        choices = ['NA', 'south', 'north']
        df_roadsection['Direction'] = np.select(conditions, choices, default='NULL')

        def nashift(x):
        #     print( pd.Series(x['Direction'].value_counts().index[0]) )
            return x['Direction'].value_counts().index[0]

        a = df_roadsection.groupby(['Driver_ID', 'Order_ID']).apply(nashift)
        df_roadsection = df_roadsection.set_index(['Driver_ID', 'Order_ID']).join(
            pd.DataFrame(a, columns=['Dir'])).reset_index().set_index(df_roadsection.index)
        df_roadsection.loc[df_roadsection['Direction']=='NA', 'Direction'] = df_roadsection.loc[
            df_roadsection['Direction']=='NA', 'Dir']
        df_roadsection = df_roadsection.drop(['Dir'], axis=1)
        
        # aggregate mean speed for each direction
        df_speed = df_roadsection[
            (df_roadsection['Direction']!='NULL') & (df_roadsection['Direction'] != 'NA')
        ].groupby(['Driver_ID', 'Order_ID', 'Direction']).apply(calculate_speed)
        df_speed = df_speed.groupby([
            pd.Grouper(key='Timestamp_begin', freq='5min'), 'Direction'])[['Speed_mean']].mean().unstack(
            )
        
        # localize to system timezone and convert to Asia/Shanghai timezone
        df_speed.index = df_speed.index.tz_localize('UTC').tz_convert(pytz.timezone('Asia/Shanghai'))
        
        # reset column label to ['Speed_north', 'Speed_south']
        df_speed.columns = ['Speed_' + df_speed.columns.levels[1][0], 'Speed_' + df_speed.columns.levels[1][1]]
        
        # write to speedtable
        speedtable_concat = pd.concat([speedtable, df_speed], sort=False)
        speedtable = speedtable_concat.groupby(speedtable_concat.index).mean()
        
        ###################
        # process density #
        ###################
        
        # compute the total count of drivers in each zone 'segment_num'
        count_info = df.loc[df['zone_interest']==0.].groupby([
            pd.Grouper(key='Timestamp', freq='5min'), 'segment_num'])['Driver_ID'].count().unstack(fill_value=0)
        
        # localize to system timezone and convert to Asia/Shanghai timezone
        count_info.index = count_info.index.tz_localize('UTC').tz_convert(pytz.timezone('Asia/Shanghai'))
        
        print('Date range processed: {} to {}'.format(count_info.index.min(), count_info.index.max()))
        count_info.columns.name = None
        datatable = datatable.add(count_info, fill_value=0)
    
    print('{}: Extraction of file {} '
          'time elapsed: {:.2f} minutes'.format(n, tf.members[0], (time.time()-start_time)/60.))

Date range processed: 2016-10-26 00:00:00+08:00 to 2016-10-27 00:00:00+08:00
1: Extraction of file <TarInfo 'xian/gps_20161026' at 0x7f02155b89a8> time elapsed: 1.17 minutes
Date range processed: 2016-10-27 00:00:00+08:00 to 2016-10-28 00:00:00+08:00
2: Extraction of file <TarInfo 'xian/gps_20161027' at 0x7f02155b85c0> time elapsed: 2.39 minutes
Date range processed: 2016-10-28 00:00:00+08:00 to 2016-10-29 00:00:00+08:00
3: Extraction of file <TarInfo 'xian/gps_20161028' at 0x7f02155b8b38> time elapsed: 3.76 minutes
Date range processed: 2016-10-29 00:00:00+08:00 to 2016-10-30 00:00:00+08:00
4: Extraction of file <TarInfo 'xian/gps_20161029' at 0x7f02155b8c00> time elapsed: 5.04 minutes
Date range processed: 2016-10-01 00:00:00+08:00 to 2016-10-02 00:00:00+08:00
5: Extraction of file <TarInfo 'xian/gps_20161001' at 0x7f02155b8cc8> time elapsed: 6.29 minutes
Date range processed: 2016-10-02 00:00:00+08:00 to 2016-10-03 00:00:00+08:00
6: Extraction of file <TarInfo 'xian/gps_20161002' at

Date range processed: 2016-11-16 00:00:00+08:00 to 2016-11-17 00:00:00+08:00
48: Extraction of file <TarInfo 'xian/gps_20161116' at 0x7f024755b4f8> time elapsed: 53.47 minutes
Date range processed: 2016-11-17 00:00:00+08:00 to 2016-11-18 00:00:00+08:00
49: Extraction of file <TarInfo 'xian/gps_20161117' at 0x7f024755b5c0> time elapsed: 54.48 minutes
Date range processed: 2016-11-18 00:00:00+08:00 to 2016-11-19 00:00:00+08:00
50: Extraction of file <TarInfo 'xian/gps_20161118' at 0x7f024755b688> time elapsed: 55.61 minutes
Date range processed: 2016-11-19 00:00:00+08:00 to 2016-11-20 00:00:00+08:00
51: Extraction of file <TarInfo 'xian/gps_20161119' at 0x7f024755b750> time elapsed: 56.89 minutes
Date range processed: 2016-11-20 00:00:00+08:00 to 2016-11-21 00:00:00+08:00
52: Extraction of file <TarInfo 'xian/gps_20161120' at 0x7f024755b818> time elapsed: 57.97 minutes
Date range processed: 2016-11-21 00:00:00+08:00 to 2016-11-22 00:00:00+08:00
53: Extraction of file <TarInfo 'xian/gps_2

In [19]:
# slice speedtable to only contain entries for 2016-10-01 00:00:00 to 2016-12-01 23:55:00
speedtable = speedtable.fillna(0.).loc['2016-10-01 00:00:00+08:00':'2016-12-01 23:55:00+08:00']

# slice datatable to only contain entries for 2016-10-01 00:00:00 to 2016-12-01 23:55:00
datatable = datatable.fillna(0.).loc['2016-10-01 00:00:00+08:00':'2016-12-01 23:55:00+08:00', '0':]

In [22]:
# finally, save datatable to csv
datatable.to_csv('datatable_full.csv')
speedtable.to_csv('speedtable_full.csv')

[Continue](02_preparation.ipynb) to next step to prepare datasets for model training